# vaccine-equity-metric

By [Sean Greene](https://www.latimes.com/people/sean-greene)

Downloads the number of vaccine doses administered by HPI quartile from a Tableau dashboard published by the California Department of Public Health.

## Import

Code formatting with [black](https://pypi.org/project/nb-black/).

In [61]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


Import dependencies.

In [62]:
import os
import pytz
from datetime import datetime
import pandas as pd

In [63]:
from tableauscraper import TableauScraper as TS

## Scrape

Set the URL

In [64]:
host = "https://public.tableau.com"

In [65]:
path = (
    "/views/COVID-19VaccineEquityMetricQuartileDashboard/VaccineEquityMetricDashboard"
)

In [66]:
url = f"{host}{path}"

In [67]:
ts = TS()
ts.loads(url)

In [68]:
dashboard = ts.getWorkbook()

Check the parameters of the dropdown menu. It should be `Partially Vaccinated` and `Fully Vaccinated`

In [69]:
parameters = dashboard.getParameters()

In [70]:
parameters[0]["values"]

['Partially Vaccinated', 'Fully Vaccinated']

In [71]:
assert len(parameters[0]["values"]) == 2

Store each one in a dataframe

In [72]:
partially_vaccinated_workbook = dashboard.setParameter(
    "Partially Vaccinated, Fully Vaccinated", "Partially Vaccinated"
)

In [73]:
for t in partially_vaccinated_workbook.worksheets:
    partial_vax_df = t.data

In [74]:
partial_vax_df = partial_vax_df.rename(
    columns={
        "Week Ending-alias": "date",
        "Hpiquartile-alias": "hpi_quartile",
        "SUM(Partially/Fully Combined)-value": "pct_partially_vaccinated",
    }
)[["date", "hpi_quartile", "pct_partially_vaccinated"]]

In [75]:
fully_vaccinated_workbook = dashboard.setParameter(
    "Partially Vaccinated, Fully Vaccinated", "Fully Vaccinated"
)

In [76]:
for t in fully_vaccinated_workbook.worksheets:
    full_vax_df = t.data

In [77]:
full_vax_df = full_vax_df.rename(
    columns={
        "Week Ending-alias": "date",
        "Hpiquartile-alias": "hpi_quartile",
        "SUM(Partially/Fully Combined)-value": "pct_fully_vaccinated",
    }
)[["date", "hpi_quartile", "pct_fully_vaccinated"]]

In [78]:
df = pd.merge(partial_vax_df, full_vax_df, how="outer", on=["date", "hpi_quartile"])

In [79]:
df

,date,hpi_quartile,pct_partially_vaccinated,pct_fully_vaccinated
0,1/9/21,4,0.284267,0.355122
1,1/16/21,4,0.315441,0.343613
2,1/23/21,4,0.333923,0.320282
3,1/30/21,4,0.316553,0.305891
4,2/6/21,4,0.300538,0.285707
5,2/13/21,4,0.306942,0.337174
6,2/20/21,4,0.307829,0.332031
7,2/27/21,4,0.290339,0.331442
8,3/6/21,4,0.282922,0.308599
9,3/13/21,4,0.262571,0.304838


## Export

Save out the data as a CSV that's datestamped to California time.

In [80]:
tz = pytz.timezone("America/Los_Angeles")

In [81]:
today = datetime.now(tz).date()

In [82]:
data_dir = os.path.join(os.path.abspath(""), "data")

In [83]:
df.to_csv(os.path.join(data_dir, f"{today}.csv"), index=False)